In [ ]:
!pip install -q torch transformers pandas sentencepiece --quiet

In [ ]:
from pathlib import Path
import pandas as pd, torch, time
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

CSV_PATH = Path(r"C:...")
# TOKENIZER_ID = Path(r"D:...")
MODEL_DIR = Path(r"D:...")
BATCH_SIZE   = 64

tok   = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_DIR, local_files_only=True, device_map="auto").eval()
device = model.device

df = pd.read_csv(CSV_PATH)
print(f"sum: {len(df):,}")

def predict(texts, batch=BATCH_SIZE):
    preds = []
    for i in tqdm(range(0, len(texts), batch), desc="bar ", unit="batch"):
        bt = tok(texts[i:i+batch], padding=True, truncation=True,
                 max_length=128, return_tensors="pt").to(device)
        with torch.no_grad():
            preds.extend(model.config.id2label[p] for p in model(**bt).logits.argmax(-1).tolist())
    return preds

t0 = time.time()
df["sentiment"] = predict(df["text"].astype(str).tolist())
print(f"done in {time.time()-t0:.1f} с")

OUT = r"C:..."
df.to_csv(OUT, index=False, encoding="utf-8-sig")
print("done :", OUT)
